In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.1 MB/s eta 0:00:00


In [2]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.7 MB/s eta 0:00:00


In [3]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.1 MB/s eta 0:00:00


In [4]:
pip install transformers optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.7 MB/s eta 0:00:00


In [5]:
import torch
from transformers import XLNetForSequenceClassification, XLNetTokenizer
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import numpy as np

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import optuna

In [8]:
model_name = "xlnet-base-cased"
tokenizer = XLNetTokenizer.from_pretrained(model_name)


In [9]:
# Load dataset
import pandas as pd
df = pd.read_csv("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/politifact_cleaned.csv")
df['SECTION_CLEANED'] = df['SECTION_CLEANED'].astype(str)

# Randomly select  % of the data
df_data = df.sample(frac=1.0, random_state=42)


# Split the dataset into training, validation, and test sets
# First, split into training and temp sets (80% training, 20% temp)
train_df, temp_df = train_test_split(df_data, test_size=0.20, random_state=42)

# Then, split the temp set into validation and test sets (50% validation, 50% test)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Optionally,reset the index of the DataFrames
train_df = train_df.reset_index(drop=True)
validation_df = validation_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


In [10]:
df.groupby("label").describe()

Unnamed: 0                                                           \
           count   mean         std    min     25%    50%     75%     max   
label                                                                       
0          432.0  839.5  124.851912  624.0  731.75  839.5  947.25  1055.0   
1          624.0  311.5  180.277564    0.0  155.75  311.5  467.25   623.0   

      word count                                                   
           count       mean       std  min  25%   50%   75%   max  
label                                                              
0          432.0  11.701389  5.222315  1.0  9.0  11.0  14.0  38.0  
1          624.0   7.931090  4.665029  1.0  5.0   7.0  10.0  53.0

In [11]:
train_df.groupby("label").describe()

Unnamed: 0                                                             \
           count     mean         std    min     25%    50%     75%     max   
label                                                                         
0          344.0  842.250  127.068042  624.0  728.75  848.0  954.25  1055.0   
1          500.0  316.502  177.314068    0.0  167.75  321.5  466.50   623.0   

      word count                                                  
           count      mean       std  min  25%   50%   75%   max  
label                                                             
0          344.0  11.56686  5.280599  1.0  9.0  11.0  14.0  38.0  
1          500.0   7.96000  4.706453  1.0  4.0   7.0  10.0  53.0

In [12]:
validation_df.groupby("label").describe()

Unnamed: 0                                                        \
           count        mean         std    min     25%    50%     75%   
label                                                                    
0           44.0  805.795455  104.407426  629.0  732.75  795.5  857.50   
1           62.0  307.677419  196.936852   39.0  113.25  297.5  499.25   

              word count                                                    
          max      count       mean       std  min  25%   50%    75%   max  
label                                                                       
0      1046.0       44.0  11.795455  5.428519  2.0  9.0  12.0  13.25  37.0  
1       622.0       62.0   8.161290  5.353698  1.0  5.0   7.5   9.00  31.0

In [13]:
test_df.groupby("label").describe()

Unnamed: 0                                                        \
           count        mean         std    min     25%    50%     75%   
label                                                                    
0           44.0  851.704545  123.172516  625.0  758.75  862.0  951.25   
1           62.0  274.983871  185.405227    6.0  126.50  251.5  421.75   

              word count                                                     
          max      count       mean       std  min   25%   50%    75%   max  
label                                                                        
0      1050.0       44.0  12.659091  4.513253  5.0  10.0  12.0  14.25  30.0  
1       618.0       62.0   7.467742  3.481651  1.0   5.0   7.0  10.00  16.0

In [14]:
df.head()

,Unnamed: 0,id,title,label,word count,SECTION_CLEANED
0,0,politifact14984,National Federation of Independent Business,1,5,National Federation Independent Business
1,1,politifact12944,comments in Fayetteville NC,1,4,comment Fayetteville NC
2,2,politifact333,"Romney makes pitch, hoping to close deal : Ele...",1,14,Romney make pitch hop close deal Elections The...
3,3,politifact4358,Democratic Leaders Say House Democrats Are Uni...,1,11,Democratic Leaders Say House Democrats Are Uni...
4,4,politifact779,"Budget of the United States Government, FY 2008",1,8,Budget United States Government FY 2008


In [15]:
train_texts = list(train_df['SECTION_CLEANED'])
len(train_texts)

844

In [16]:
train_labels = list(train_df['label'])
len(train_labels)

844

In [17]:
train_df.head(10)

,Unnamed: 0,id,title,label,word count,SECTION_CLEANED
0,262,politifact1825,'This Week' Transcript: Kaine and Steele,1,6,This Week Transcript Kaine Steele
1,816,politifact14085,BREAKING: Bill O’Reilly In Critical Condition ...,0,12,BREAKING Bill OReilly In Critical Condition Af...
2,538,politifact6998,The full transcript of the third presidential ...,1,8,The full transcript third presidential debate
3,495,politifact9442,a director of research for the foreign policy ...,1,13,director research foreign policy program Brook...
4,266,politifact8172,Remarks by the President on Responsible Homeow...,1,7,Remarks President Responsible Homeownership
5,620,politifact329,Change We Can Believe In,1,5,Change We Can Believe In
6,688,politifact13982,Kasich: “If A Woman Is Not Intelligent Enough ...,0,22,Kasich “If A Woman Is Not Intelligent Enough T...
7,148,politifact6603,"Ad says Obama apologized, showed weakness on Iran",1,8,Ad say Obama apologize show weakness Iran
8,702,politifact13038,The numbers don't lie... - Occupy Democrats,0,7,The number dont lie Occupy Democrats
9,887,politifact15416,Attorney General Sessions Statement on Recusal,0,6,Attorney General Sessions Statement Recusal


In [18]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, tokenized_data, labels):
        self.input_ids = tokenized_data["input_ids"]
        self.attention_mask = tokenized_data["attention_mask"]
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

In [19]:
# Tokenize and format data
train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt", max_length=128)
#tokenized_data = tokenizer(train_texts, padding=True, truncation=False, return_tensors="pt")
train_labels = torch.tensor(train_labels)
train_dataset = CustomDataset(train_encodings, train_labels)

<ipython-input-18-91207c483471>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels)


In [20]:
# Create a PyTorch dataset for validation
from torch.utils.data import Dataset

class CustomValidationDataset(Dataset):
    def __init__(self, tokenized_data, labels):
        self.input_ids = tokenized_data["input_ids"]
        self.attention_mask = tokenized_data["attention_mask"]
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

In [21]:
val_texts = list(validation_df['SECTION_CLEANED'])
len(val_texts)

106

In [22]:
val_labels = list(validation_df['label'])
len(val_labels)

106

In [23]:
# Tokenize and format data
val_encodings = tokenizer(val_texts, padding=True, truncation=True, return_tensors="pt", max_length=128)

val_labels = torch.tensor(val_labels)
validation_dataset = CustomValidationDataset(val_encodings, val_labels)

<ipython-input-20-409cfb690dd2>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels)


In [24]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./XLNET/TrainingArgs/",
    evaluation_strategy="steps",  # Evaluate every `eval_steps`
    eval_steps=75,  # Adjust as needed
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./XLNET/Logs/",
    logging_steps=75,
    learning_rate=5e-5,
    num_train_epochs=2
)

In [25]:
# Define a function for training and evaluation
def train_and_evaluate(trial):
    # Define the hyperparameters to optimize

    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 15)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-3, 1e-1)
    warmup_steps = trial.suggest_int("warmup_steps", 100, 1000)

    # Update the training arguments
    training_args.num_train_epochs = num_train_epochs
    training_args.per_device_train_batch_size = per_device_train_batch_size
    training_args.weight_decay = weight_decay
    training_args.warmup_steps = warmup_steps
    training_args.learning_rate = learning_rate


    model_name = "xlnet-base-cased"
    model = XLNetForSequenceClassification.from_pretrained(model_name)

    # Create a Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,

    )

    # Train the model
    trainer.train()

    # Evaluate the model
    results = trainer.evaluate(validation_dataset)
    print("Evaluation Results:", results)

    model = 45
    return results["eval_loss"]


In [26]:
# Create an Optuna study
study = optuna.create_study(direction="minimize")


[I 2023-10-18 13:36:18,105] A new study created in memory with name: no-name-6c3ef076-01d2-498c-ad17-320ec9070cc8


In [27]:
# Optimize hyperparameters
study.optimize(train_and_evaluate, n_trials=15)

<ipython-input-25-cb3bebbc9129>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)
<ipython-input-25-cb3bebbc9129>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-3, 1e-1)


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
75,0.651000,0.497691
150,0.531800,0.378132


[I 2023-10-18 13:37:22,353] Trial 0 finished with value: 0.39746806025505066 and parameters: {'num_train_epochs': 4, 'per_device_train_batch_size': 16, 'learning_rate': 1.0962629214103148e-05, 'weight_decay': 0.002609289134934085, 'warmup_steps': 231}. Best is trial 0 with value: 0.39746806025505066.


Evaluation Results: {'eval_loss': 0.39746806025505066, 'eval_runtime': 0.3375, 'eval_samples_per_second': 314.048, 'eval_steps_per_second': 41.478, 'epoch': 4.0}


...

Evaluation Results: {'eval_loss': 0.31697601079940796, 'eval_runtime': 0.356, 'eval_samples_per_second': 297.743, 'eval_steps_per_second': 39.325, 'epoch': 7.0}


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
75,0.697200,0.618321
150,0.632600,0.492135
225,0.518100,0.410641
300,0.462000,0.365375
375,0.403100,0.325773
450,0.305200,0.402644


[I 2023-10-18 14:11:44,928] Trial 14 finished with value: 0.36914217472076416 and parameters: {'num_train_epochs': 9, 'per_device_train_batch_size': 16, 'learning_rate': 1.4046555448127469e-05, 'weight_decay': 0.005226976941648819, 'warmup_steps': 818}. Best is trial 13 with value: 0.31697601079940796.


Evaluation Results: {'eval_loss': 0.36914217472076416, 'eval_runtime': 0.3461, 'eval_samples_per_second': 306.308, 'eval_steps_per_second': 40.456, 'epoch': 9.0}


In [28]:
# Get the best hyperparameters
best_params = study.best_params
best_num_train_epochs = best_params["num_train_epochs"]
best_batch_size = best_params["per_device_train_batch_size"]
best_learning_rate = best_params["learning_rate"]
best_weight_decay = best_params["weight_decay"]
best_warmup_steps = best_params["warmup_steps"]
# Print the best hyperparameters
print(f"Best Number of Epochs: {best_num_train_epochs}")
print(f"Best Batch Size: {best_batch_size}")
print(f"Best Learning Rate: {best_learning_rate}")
print(f"Best Weight Decay: {best_weight_decay}")
print(f"Best Warmup Steps: {best_warmup_steps}")

Best Number of Epochs: 7
Best Batch Size: 16
Best Learning Rate: 1.3617893750625895e-05
Best Weight Decay: 0.001047076633830012
Best Warmup Steps: 992
